## Preparations

In [2]:
import sys
from pyspark import SparkContext

# start spark with 4 worker threads (4 cores available)
sc = SparkContext("local[4]")
sc.setLogLevel("ERROR")

# read the input files into an RDD[String] and split each line into an array of items
# 1 file of machine events (~ 3 MB)
# first 200 files of job events (~ 128 MB)
# first 7 files of task events (~ 137 MB)
# first file of task usage (~ 364 MB)
machine_events_file = sc.textFile("data/machine_events/part-00000-of-00001.csv").map(lambda x : x.split(','))
job_events_file = sc.textFile("data/job_events/part-00[0-1][0-9][0-9]-of-00500.csv").map(lambda x : x.split(','))
task_events_file = sc.textFile("data/task_events/part-0000[0-6]-of-00500.csv").map(lambda x : x.split(','))
task_usage_file = sc.textFile("data/task_usage/part-00001-of-00500.csv").map(lambda x : x.split(','))

# keep the RDD in memory
machine_events_file.cache()
job_events_file.cache()
task_events_file.cache()
task_usage_file.cache()


PythonRDD[11] at RDD at PythonRDD.scala:53

In [2]:
# show the number of observations (lines)
print(machine_events_file.count())
print(job_events_file.count())
print(task_events_file.count())
print(task_usage_file.count())


37780
782821
1240991
2352447


## Question 1: What is the distribution of the machines according to their CPU capacity?

To answer this question we first prepare the indexes `(machine ID and CPU)`. Then from the machine_events_file we take only columns that correspond to those indexes. Finally we apply `distinct()` method to remove reapeating lines.

In [2]:
# prepare indexes (machine ID and CPU) according to "schema.csv" file
i_mID = 1 # machine ID
i_CPU = 4 # CPU

# create (machine_ID, CPU) unique pairs
mID_CPU = machine_events_file.map(lambda x: (x[i_mID], x[i_CPU])).distinct()

# TODO: create graph (or other illustr)


## Question 2: What is the percentage of computational power lost due to maintenance (a machine went offline and reconnected later)?

We need to calculate $\frac{lost \; CPU \; power}{total \; CPU \; power}$x100. So first we calculate the total CPU power, then the total lost CPU power, and apply the formula. We can do the same for the memory capacity.

By looking at the results we can conclude that there is more loss of memory capacity than the loss of computational power.

In [3]:
# get total CPU power
col_CPU = machine_events_file.map(lambda x: x[4])  # get only CPU column
col_CPU = col_CPU.filter(lambda x: len(x) > 0)     # filter rows with no CPU data 
col_CPU = col_CPU.map(lambda x: float(x))          # convert CPU to float
total_CPU = col_CPU.sum()                          # sum up column

# get total lost CPU power (event_type == 1)
col_CPU_lost = machine_events_file.map(lambda x: (x[4], x[2])).filter(lambda x: x[1] == '1')
col_CPU_lost = col_CPU_lost.filter(lambda x: len(x[0]) > 0).map(lambda x: float(x[0]))
total_lost_CPU = col_CPU_lost.sum()

# result
print("Percentage of computational power lost due to maintenance:", 
      round((total_lost_CPU / total_CPU) * 100, 2), "%")

# same for % of memory capacity lost
total_mem = machine_events_file.map(lambda x: x[5]).filter(lambda x: len(x) > 0).map(lambda x: float(x)).sum()
total_lost_mem = machine_events_file.map(lambda x: (x[5], x[2])).filter(lambda x: x[1] == '1' and len(x[0]) > 0
                                                                       ).map(lambda x: float(x[0])).sum()

print("Percentage of memory capacity lost due to maintenance:", 
      round((total_lost_mem / total_mem) * 100, 2), "%")

Percentage of computational power lost due to maintenance: 23.99 %
Percentage of memory capacity lost due to maintenance: 24.47 %


## Question 3: What is the distribution of the number of jobs/tasks per scheduling class?

The goal here is to create two types of tuples: `(sch_class, nb_jobs)` and `(sch_class, nb_tasks)`. 

In [4]:
# index of scheduling class is 5 at job_events_file and 7 at task_events_file
# create (sch_class, 1) tuple
# and reduce by sch_class, summing up values
jobs_per_class =   job_events_file.map(lambda x: (x[5], 1)).reduceByKey(lambda x,y: x+y)
tasks_per_class = task_events_file.map(lambda x: (x[7], 1)).reduceByKey(lambda x,y: x+y)

# show results
print(jobs_per_class.collect())
print(tasks_per_class.collect())

# TODO: bar chart

[('1', 257814), ('0', 305793), ('2', 211509), ('3', 7705)]
[('1', 155637), ('0', 805914), ('3', 60468), ('2', 218972)]


## Question 4: What is the percentage of jobs/tasks that got killed or evicted depending on the scheduling class?

The goal is to create two types of tuples:

`(sch_class, percentage_of_jobs_evicted, percentage_of_jobs_killed)`

and

`(sch_class, percentage_of_tasks_evicted, percentage_of_tasks_killed)`

The first step is to create `(sch_class, 1)` pairs for both jobs and tasks and for event types `EVICT` and `KILL`. Then we reduce the results by key, summing up the values, to get the total per class. We also apply `collectAsMap()` method, for the future operations: it will allows us to easily get the value, by just putting the key as an index. 

As a next step we calculate the total number of events per class, by creating a tuples `(event_type, 1)` and then reducing by key and summing up the values.

Now we create two types of pairs: `(sch_class, percentage_of_evict)` and `(sch_class, percentage_of_kill)` for both jobs and tasks, where the keys are the scheduling classes and values are the % of evicted/killed jobs/tasks. The last ones are `0.0` if the scheduling class has no task/job evicted or killed. Otherwise the value will be $\frac{number \; of \; jobs/tasks \; evicted \; or \; killed\; for\; a\; class}{total \;number\; of\; jobs/tasks}$x100  

We also transform some variable to a dictionary, in order to simplify the access to the value of by just providing the key as an index. This will be the merging process for the final result. 

For the final step we prepare our goal tuples. The first and the second elements for the result we get by directly applying the relevant indexes (0 and 1), since we are applying the `map()` method on the variable where those elements exit. The third element for the tuple has to be taken from the dictionary that was prepared during the previous step.

For this question we can conclude that there are no evicted jobs for the scheduling classes `2` and `3`.

In [5]:
# for event_type, index 2 is EVICT, index 5 is KILL
# event_type index is 3 at job_events and 5 at task_events

# create (sch_class, 1) pairs, for jobs and tasks, for event_type 2 and 5
jobs_evict = job_events_file.filter(lambda x: x[3] == '2').map(lambda x: (x[5], 1))
jobs_kill  = job_events_file.filter(lambda x: x[3] == '5').map(lambda x: (x[5], 1))
tasks_evict = task_events_file.filter(lambda x: x[5] == '2').map(lambda x: (x[7], 1))
tasks_kill  = task_events_file.filter(lambda x: x[5] == '5').map(lambda x: (x[7], 1))

# reduce by sch_class, summing up values, and tranform to dictionary
jobs_evict_per_class   = jobs_evict.reduceByKey(lambda x,y: x+y).collectAsMap()
jobs_kill_per_class     = jobs_kill.reduceByKey(lambda x,y: x+y).collectAsMap()
tasks_evict_per_class = tasks_evict.reduceByKey(lambda x,y: x+y).collectAsMap()
tasks_kill_per_class   = tasks_kill.reduceByKey(lambda x,y: x+y).collectAsMap()

# total nb of events per class
job_events_per_class = job_events_file.map(lambda x: (x[5], 1)).reduceByKey(lambda x,y: x+y)
task_events_per_class = task_events_file.map(lambda x: (x[7], 1)).reduceByKey(lambda x,y: x+y)

# (sch_class, percentage_of_evict) and (sch_class, percentage_of_kill)
job_percent_evict_per_class = job_events_per_class.map(lambda x: (x[0], (jobs_evict_per_class[x[0]] / x[1])*100 
                                                                          if x[0] in jobs_evict_per_class
                                                                          else 0.0))
job_percent_kill_per_class  = job_events_per_class.map(lambda x: (x[0], (jobs_kill_per_class[x[0]] / x[1])*100 
                                                                          if x[0] in jobs_kill_per_class
                                                                          else 0.0))
task_percent_evict_per_class = task_events_per_class.map(lambda x: (x[0], (tasks_evict_per_class[x[0]] / x[1])*100 
                                                                          if x[0] in tasks_evict_per_class
                                                                          else 0.0))
task_percent_kill_per_class  = task_events_per_class.map(lambda x: (x[0], (tasks_kill_per_class[x[0]] / x[1])*100 
                                                                          if x[0] in tasks_kill_per_class
                                                                          else 0.0))

# transform to dictionary, need for merging
job_percent_kill_per_class = job_percent_kill_per_class.collectAsMap()
task_percent_kill_per_class = task_percent_kill_per_class.collectAsMap()

# (sch_class, %_jobs_evicted, %_jobs_killed)
res_job = job_percent_evict_per_class.map(lambda x: (x[0], x[1], job_percent_kill_per_class[x[0]]))
# (sch_class, %_tasks_evicted, %_tasks_killed)
res_task = task_percent_evict_per_class.map(lambda x: (x[0], x[1], task_percent_kill_per_class[x[0]]))

# print results
for f in sorted(res_job.collect()):
    print(f)
for f in sorted(res_task.collect()):
    print(f)
    
# TODO: round(x, 4), double bar chart

('0', 0.0032701860408838658, 17.661620769605584)
('1', 0.0015515061245704267, 6.230460719743691)
('2', 0.0, 14.667933752228038)
('3', 0.0, 24.87994808565866)
('0', 4.828058576969751, 5.942569554567857)
('1', 5.226906198397553, 1.477155175183279)
('2', 2.8410025026030725, 16.00798275578613)
('3', 0.3274459218098829, 2.39630879142687)


## Question 5: Do tasks with low priority have a higher probability of being evicted?

The goal is to create `(task_priority, percentage_of_evicted)` pairs. 

As a fist step we create `(task_priority, 1)` pairs, then we reduce them by key, summing up the values.

Then we do the same operation, but only for the rows where `event == EVICT`. Then we tranform this result to a (key-value) dictionaty, to simplify the access as explained above.

Now we are ready to create the target pair, where the first element is the *task priority* and the second one is 0.0 if for the *task priority* there was never an eviction. Otherwise $\frac{nb\; of\; evictions\; for\; a\; priority}{total\; nb\; of\; tasks\; for\; a\; priority}$x100.

By seeing the results, we can conclude that the tasks with low priority have a higher probability of being evicted.

In [6]:
# task_priority index is 8

# create (task_priority, 1) pairs, and reduce by task_priority, summing up values
total_by_priority = task_events_file.map(lambda x: (x[8], 1)).reduceByKey(lambda x,y: x+y)

# create (task_priority, 1) pairs, where event == EVICT, and reduce by task_priority, summing up values
nb_evicted_by_priority = task_events_file.filter(lambda x: x[5] == '2').map(lambda x: (x[8], 1)
                                                                           ).reduceByKey(lambda x,y: x+y)

# convert to dictionary
nb_evicted_by_priority = nb_evicted_by_priority.collectAsMap()

res = total_by_priority.map(lambda x: (x[0], round((nb_evicted_by_priority[x[0]] / x[1])*100, 2) 
                                               if x[0] in nb_evicted_by_priority 
                                               else 0.0))

# show result
res.sortBy(lambda x: int(x[0])).collect()


# TODO: illustrate

[('0', 11.12),
 ('1', 1.96),
 ('2', 0.07),
 ('8', 0.06),
 ('9', 0.15),
 ('10', 0.1),
 ('11', 0.0)]

## Question 6: In general, do tasks from the same job run on the same machine?

For this question we are going to prepare a pair, where the first element is `job_ID` and the second one is `highest % of tasks from that job_ID running on same machine`.

If the second element is greater or equal to 50% in at least half of cases (job_IDs), then answer for this question will be positive.

As a first step we apply the `map()` method to get `(job_ID, machine_ID)` pairs, and `groupByKey().mapValues(list)` methods to get `(job_ID, list_of_machines)`.

Now we create our target tuple `(job_ID, highest % of tasks from that job_ID running on same machine)`. To get the second element for this tuple, we take `list_of_machines`, we find the most frequent element in this list, we count the number of appearences of that element in the list, and then we divide that number by the length of the list and multiply by 100 to get the percentage.

Finally we print the result.

In [10]:
# job ID index is 2, machine ID is 4

# create (job_ID, list_of_machines) pair
job_machine = task_events_file.map(lambda x: (x[2], x[4])).groupByKey().mapValues(list)

# create (job_ID, highest % of tasks from that job_ID running on same machine) pair
highest_percentage_per_job = job_machine.map(lambda x: (x[0], 
                                                        (x[1].count(max(set(x[1]), key=x[1].count))
                                                        / len(x[1])) * 100))

# keep the RDD in memory, because computation is very costly
job_machine.cache()
highest_percentage_per_job.cache()

# total nb of jobs
total_per_job = highest_percentage_per_job.count()
# nb of jobs where % of tasks running on same machine is >= 50
half_or_more_per_job = highest_percentage_per_job.filter(lambda x: x[1] >= 50.0).count()

# result
print("In", round((half_or_more_per_job / total_per_job) * 100, 2), 
      "% of jobs, at least 50 % of tasks run on the same machine")
print("We can conclude that in general, tasks from the same job run on the same machine")


In 74.09 % of jobs, at least 50 % of tasks run on the same machine
We can conclude that in general, tasks from the same job run on the same machine


## Question 7: Are there tasks that consume significantly less resources than what they requested?

In this question to identify a task we will use `(job_ID, task_index)` pair. As a ressource to check we'll start with CPU power. 

The answer to this question will be positive, if we can prove two things:
- that 0 CPU was never requested
- that 0 CPU was used at least once

As a result we can see that sometimes CPU is not used at all.

In [2]:
# task_event columns:
#  job ID (2), 
#  task index (3), 
#  CPU request (9)

# task_usage columns:
#  job ID (2), 
#  task index (3),
#  sampled CPU usage (19)

# create tuple ((job_ID, task_index), CPU)
#
# where (job_ID, task_index) is a key
#       CPU is a value
#
# and filter rows where no info about resource
requested_pow = task_events_file.map(lambda x: ((x[2], x[3]), x[ 9])).filter(lambda x: len(x[1]) > 0).distinct()
used_pow      =  task_usage_file.map(lambda x: ((x[2], x[3]), x[19])).filter(lambda x: len(x[1]) > 0).distinct()

# convert CPU to float
requested_pow = requested_pow.map(lambda x: (x[0], float(x[1])))
used_pow = used_pow.map(lambda x: (x[0], float(x[1])))

# return False if 0 CPU power was never requested: 
zero_requested = requested_pow.filter(lambda x: x[1] == 0)
print(zero_requested.isEmpty())

# return True if 0 CPU power was ever used
zero_used = used_pow.filter(lambda x: x[1] == 0)
print(not zero_used.isEmpty())

# get an example of a task that consumed significantly less than requested
task = zero_used.take(1)[0][0]
used = zero_used.take(1)[0][1]
requested = requested_pow.lookup(task)

print("The task having (job ID, task index) as", task, 
      "has requested", requested[0], "amount of CPU, but used", used)

False
True
The task having (job ID, task index) as ('6251812952', '1761') has requested 0.02499 amount of CPU, but used 0.0


## Question 8: How often does it happen that the resources of a machine are over-committed?

As a ressource for this question we are going to consider CPU power. We will compute $\frac{a}{b}$ x100, where $a$ is the number of times when `(machine_ID, CPU_requested)` == `(machine_ID, CPU_used)`. And $b$ is the number of machines occuring in both `task_events` and `task_usage` files, where the last one is filtered to have the `end time` <= `time` of the last row of the filtered `task_events` file.

In [31]:
# filter rows with empty machine ID and CPU
file_event = task_events_file.filter(lambda x: len(x[4]) > 0 and
                                               len(x[9]) > 0)
# get `time` of the last row
time = file_event.map(lambda x: x[0]).collect()[-1]

# get (machine_ID, CPU_requested) distinct pairs
CPU_requested = file_event.map(lambda x: (x[4], x[9])).distinct()

# get (machine_ID, CPU_used) pairs, where end_time <= `time`
# and rows are with non-empty machine ID and CPU
file_usage = task_usage_file.filter(lambda x: int(x[1]) <= int(time)
                                                  and len(x[4])  > 0
                                                  and len(x[19]) > 0)

CPU_used = file_usage.map(lambda x: (x[4], x[19])).distinct()

# get the nb of over-committements
# i.e. nb of times when (machine_ID, CPU_requested) == (machine_ID, CPU_used)
over_committed = CPU_requested.intersection(CPU_used).count()

# get the total number of machines occuring in both files
machines_requested = CPU_requested.map(lambda x: x[0])
machines_used = CPU_used.map(lambda x: x[0])
total_machines = machines_requested.intersection(machines_used).count()

# print the result
print("Approximately", round(over_committed/total_machines * 100, 2), 
      "% of times it happen that the resources of a machine are over-committed")

Approximately 35.03 % of times it happen that the resources of a machine are over-committed


## Conclusions

One of the main challenges of this work were:

- to think in key-value paradigm, where the tuples consist of only two elements. It would allow to more efficiently use the pyspark API.

- to imagine how the final result should look like, because there may be different ways of representations, and it's better to choose the simplest one. But for some cases the simplest representation is less convinient. For example, for the question 2, initial goal for the result was to create `(time, % of lost CPU)` pair, to  be able to track the % of the loss with respect to time. But it turned out that the second element of that tuple will always be 0% or 100%, which wouldn't be nice to visualize.

- to deal with errors which are generated because of the missing data (e.g. for some rows `CPU` is not given).

- to identify the columns linking the same data in two data files. For example, for a task in a `task_usage` file, how to find that same task in `task_events` file. Start/end times are not relevant for unique identification, according to documentation. Task index is not unique. The combination `(job ID, task index, machine ID)` is also not unique.